In [1]:
import pandas as pd
import os
import time
import openai
import pandas as pd
import warnings
import re

warnings.filterwarnings('ignore')

PATH_data_export = "/Users/huiziyu/Library/CloudStorage/GoogleDrive-huiziy@g.ucla.edu/My Drive/Project - LLM Bib in Biomedical & Health/data/processed"
df = pd.read_csv(f'{PATH_data_export}/relevant_data_gpt.csv')

In [16]:
def gpt_prompt(text):
    prompt = """You are a public health and medical policy analyst.
    Please provide the names of disease, disorder, health symptom, public health crisis, and diagnostic tests in the following text.
    Please only provide all the names without reasoning.
    If there is no such name, please say "None mentioned".
    The text is:
    """
    result = prompt+"\n"+text
    return result

In [17]:
from openai._client import OpenAI

SECRET_FILE = '../../secrets.txt'
with open('../../secrets.txt') as f:
    lines = f.readlines()
    for line in lines:
        if line.split(',')[0].strip() == "open_ai_key":
            open_ai_key = line.split(',')[1].strip()

openai_client = OpenAI(api_key=open_ai_key)

# GPT models (GPT-3.5 and GPT-4)
def run_gpt(text_prompt, max_tokens_to_sample: int = 3000, temperature: float = 0, client=openai_client, model = "gpt-3.5-turbo"):
    # use gpt-3.5-turbo unless specify gpt-4
    response = client.chat.completions.create(
      model = model, 
      messages=[
        {"role": "user", "content": text_prompt},
      ],
      temperature=temperature,
      max_tokens=max_tokens_to_sample
    )
    return response.choices[0].message.content

In [32]:
ID = 0
results_df = pd.DataFrame(columns=['ID', 'Original_Text', 'GPT_Results'])
for text in df.Title_abstract:
    #print(text)
    text_prompt = gpt_prompt(text)
    if len(text_prompt) > 2000:
        text_prompt = text_prompt[:2000]
    result = run_gpt(text_prompt)
    results_df = results_df._append({'ID': ID, 'Original_Text': text, 'GPT_Results': result}, ignore_index=True)
    ID += 1

In [33]:
results_df["GPT_Results"]

0       disease: None mentioned\ndisorder: None mentio...
1                                         None mentioned.
2       disease: None mentioned\ndisorder: None mentio...
3       Names of disease: None mentioned\nNames of dis...
4       Disease: None mentioned\nDisorder: None mentio...
                              ...                        
1442                                      None mentioned.
1443    Disease: musculoskeletal disorders, lower back...
1444    Disease: None mentioned\nDisorder: None mentio...
1445    Disease: Epilepsy\nDisorder: None mentioned\nH...
1446    disease: mental health disorders\ndisorder: me...
Name: GPT_Results, Length: 1447, dtype: object

In [34]:
import pandas as pd
import re

def extract_entities(text, entity_name):
    """
    Extracts entities (like Disease, Disorder, etc.) from the text.
    
    Parameters:
    text (str): The text from which to extract the entity.
    entity_name (str): The name of the entity (e.g., "Disease").

    Returns:
    str: The extracted entity or 'None mentioned' if not found.
    """
    pattern = fr"{entity_name}: (.*?)\n"
    match = re.search(pattern, text)
    return match.group(1) if match else "none mentioned"

# Assuming df is your DataFrame with a column 'Original_Text'
entities = ['disease', 'disorder', 'health symptom', 'public health crisis', "diagnostic tests"]

# Applying the function to each row and entity
for entity in entities:
    results_df[entity] = results_df['GPT_Results'].apply(lambda text: extract_entities(text.lower(), entity))

In [38]:
results_df.columns = [column.replace(' ', '_') for column in results_df.columns]
results_df.disease.value_counts()

disease
none mentioned                                                                  1149
covid-19                                                                          36
cancer                                                                            12
depression                                                                         6
breast cancer                                                                      6
                                                                                ... 
non-communicable diseases, obesity, cardiovascular diseases, type-2 diabetes       1
headache                                                                           1
cardiovascular disease                                                             1
liver disease, hepatitis b, hepatocellular carcinoma (hcc)                         1
infectious diseases                                                                1
Name: count, Length: 195, dtype: int64

In [37]:
results_df.disease.value_counts()

disease
none mentioned                                                                  1149
covid-19                                                                          36
cancer                                                                            12
depression                                                                         6
breast cancer                                                                      6
                                                                                ... 
non-communicable diseases, obesity, cardiovascular diseases, type-2 diabetes       1
headache                                                                           1
cardiovascular disease                                                             1
liver disease, hepatitis b, hepatocellular carcinoma (hcc)                         1
infectious diseases                                                                1
Name: count, Length: 195, dtype: int64

In [36]:
results_df.to_csv(f'{PATH_data_export}/disease_extraction_gpt_results.csv', index=False)
results_df.to_excel(f'{PATH_data_export}/disease_extraction_gpt_results.xlsx', index=False)